<a href="https://colab.research.google.com/github/juancamiloespana/RL_DS/blob/master/RL_knapsack.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Descripción del problema de la mochila

Considere el caso en el cual usted dispone de una mochila con una capacidad máxima $b$, debiendo decir dentro de un conjunto de objetos, cada uno con una peso y valor dado, cuales debe empacar con el fin de maximizar el valor de los objetos en la mochila. La siguiente tabla presenta los datos para una instancia del problema en el que se asume que la mochila tiene una capacidad de 100 unidades

| Item  | 1  | 2  | 3  | 4  | 5  | 6  | 7  | 8 | 9  | 10 | 11 | 12 | 13 | 14 | 15 |
|-------|----|----|----|----|----|----|----|---|----|----|----|----|----|----|----|
| Valor | 15 | 80 | 90 | 60 | 40 | 15 | 95 | 8 | 45 | 50 | 60 | 35 | 32 | 27 | 61 |
| peso  | 7  | 20 | 25 | 18 | 15 | 5  | 30 | 4 | 14 | 17 | 19 | 12 | 12 | 10 | 17 |

<center>¿Cuáles items deberán empacarse en la mochila?</center>

---

Este problema puede formularse de la siguiente forma

## Conjuntos
> $I$: Conjunto de items

## Parámetros
> $v_{i}$: Valor de cada item $i$ 

> $w_i$: peso de cada item $i$

> $C$: Capacidad de la mochila

## Variables de decisión

> \begin{equation}
    x_i=
    \begin{cases}
      1, & \text{si se selecciona el item i} \\
      0, & \text{en otro caso}
    \end{cases}
  \end{equation}

## Función objetivo

> $$\text{maximizar}\  \sum_{i \in I}v_{i}x_{i} $$

## Restricciones

**1. Capacidad de la mochila**

> $\sum_{i \in I}  w_ix_i \leq C$


**2. restricción de dominio de las variables**

Expresa que todas las variables deben ser no negativas. Es decir:
> $x_{i} \in \{0,1\} \ \forall i \in I$ 

# Datos de entrada

In [ ]:
#ITEMS = {1, 2, 3, 4, 5, 6, 7,	8, 9, 10, 11, 12, 13, 14, 15},
#profits = {1: 15, 2: 80, 3: 90 , 4: 60 , 5: 40, 6: 15, 7: 95,	8: 8, 
#                   9: 45, 10: 50, 11: 60, 12: 35, 13: 32, 14: 27, 15: 61}
#weights = {1: 7, 2: 20, 3: 25 , 4: 18 , 5: 15, 6: 5, 7: 30,	8: 4, 
#                   9: 14, 10: 17, 11: 19, 12: 12, 13: 12, 14: 10, 15: 17}
#capacity = 100

ITEMS = [1, 2, 3, 4, 5, 6, 7,	8, 9, 10, 11, 12, 13, 14, 15]
n_items = len(ITEMS)
values = [15, 80, 90, 60, 40, 15, 95, 8, 45, 50, 60, 35, 32, 27, 61]
weights = [7, 20, 25, 18, 15, 5, 30, 4, 14, 17, 19, 12, 12, 10, 17]
capacity = 100

# Modelo de aprendizaje reforzado 


El modelo formulado considera los siguientes elementos
* Un entorno (`environment`) del que en cada instante del tiempo se puede obtener una observación (`obs`) de su estado actual. 
* Una observación corresponde a un vector de `n_items + 2` posiciones, donde `n_items` representa el número de items disponibles para empacar en la mochila y las dos posiciones adicionales corresponden, respectivamente, al valor de los items actuelmente empacados y a la capacidad remanente en la mochila . Un ejemplo de la observación del sistema para un problema con 5 items sería: 
> `[ 0 1 0 1 0 17 20]`
* Un agente (`agent`) que con base en una observación del estado actual del sistema toma una acción `a` respecto a que item de los que no han sido empacados debe adicionarse a la mochila
* La decisión del agente se basa en la predicción realizada por una red neuronal (`net`). Esta red recibe una observación y devuelve la probabilidad con que cada item deberia ser seleccionado



Implementaremos cada uno de los elementos mencionados, para ello primero instalamos los pquetes requeridos

Instalar librerias y paquetes

In [ ]:
from collections import namedtuple
import numpy as np

import torch
import torch.nn as nn
import torch.optim as optim

## Red neuronal para predecir acción 

In [ ]:
class Net(nn.Module):
    def __init__(self, obs_size, hidden_size, n_actions):
        super(Net, self).__init__()
        self.net = nn.Sequential(
            nn.Linear(obs_size, hidden_size),
            nn.ReLU(),
            nn.Linear(hidden_size, n_actions)
        )

    def forward(self, x):
        return self.net(x)

Probemos la red con una observación fictica

In [ ]:
# Definimos la estructura interna de la red
HIDDEN_SIZE = 128

# Creamos la red, definimos el tipo de oprimización y suavizamos la salida
net = Net(n_items+2, HIDDEN_SIZE, n_items)
objective = nn.CrossEntropyLoss()
optimizer = optim.Adam(params=net.parameters(), lr=0.01)
sm = nn.Softmax(dim=1)

# Creamos una observación cualquiera y calculamos las probabilidades de cada item para ser empacado
obs = [1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 12, 85]
obs_v = torch.FloatTensor([obs])
act_probs_v = sm(net(obs_v))
act_probs_v
act_probs = act_probs_v.data.numpy()[0]
act_probs


## Clase `environment`

In [ ]:
class environment:
  '''    
    Attributes:
    n (int): number of items 
    capacity (real): knapsck capcity
    values (list:int): Value of each item
    weights (list:int): Weight of each item
    iters (int): Number of iterations in each try to solve the problem
                 it can be set equal to the number of items

  '''

  def __init__(self, n, capacity, values, weights, iters):
        
        self.n_items = n # number of items
        self.decision = np.zeros(self.n_items)   # 1 for items selected
        self.values = values # Value of each item
        self.weights = weights  # Weight of each item            
        self.iters = iters # Number of iterations        
        self.n_moves = 0    # Search steps counter        
        self.kp_value = 0 # Total knapsack value
        self.capacity = capacity # knapsck remanent capacity
        self.obs = self.decision.tolist() # observation 
        self.obs.append(self.kp_value)
        self.obs.append(self.capacity)
        
  def step(self, a):
    """ Takes an action and updates the environment. """
    is_done = False # Indicates if the limit of iterations is reached
    step_valid = False # Indicates if an action is feasible     
    self.n_moves += 1 # update number of tried moves

    # update decisions vector if the movement is feasible
    if self.decision[a] == 0 and (self.weights[a] <= self.capacity):
      step_valid = True
      self.decision[a] = 1
      self.kp_value += self.values[a] 
      self.capacity -= self.weights[a]

      # Update observation
      self.obs = self.decision.tolist() 
      self.obs.append(self.kp_value)
      self.obs.append(self.capacity)

    else:
      is_done = True  
     
    # Verify if the limit of iterations is reached
    if self.n_moves >= self.iters:
      is_done = True   
        
    return step_valid, is_done
    
  def reset(self):
    """ Resets results while keeping settings"""
    self.decision = np.zeros(self.n_items) 
    self.n_moves = 0 
    self.kp_value = 0
    self.capacity = capacity
    self.obs = self.decision.tolist() 
    self.obs.append(self.kp_value)
    self.obs.append(self.capacity)

## Clase `agent`

In [ ]:
class agent:
  '''    
    Attributes:
    sm (Softmax): softmax operator 
    eps (real [0,1]): controls diversity
    values (list:int): Value of each item
    weights (list:int): Weight of each item
    iters (int): Number of iterations in each try to solve the problem
                 it can be set equal to the number of items

  '''
  def __init__(self, sm, eps):    
    self.action = -99 # Define null action
    self.sm = sm    
    self.eps = eps


  def choose_action(self, env, net):
    '''Obtains and observation from the environment and chooses and 
    action based on the probabilities given for a neural network'''    
    p = np.random.rand() # Generate random number
    # Takes firs decision randomly 
    if env.decision.sum() == 0:      
      self.action = np.random.choice(env.n_items)
    elif p < self.eps:
      # Randomly select an action
      self.action = np.random.choice(env.n_items)
    else:
      # Take greedy action chosing the one with highest probability
      obs_v = torch.FloatTensor([env.obs])
      act_probs_v = self.sm(net(obs_v))
      act_probs = act_probs_v.data.numpy()[0]      
      self.action = np.random.choice(len(act_probs), p=act_probs)      
    
    return self.action



## Iteración en batches

In [ ]:
# usa namedtuples para guardar los pasos de cada episodio y los episodios 
EpisodeStep = namedtuple('EpisodeStep', field_names=['observation', 'action'])
Episode = namedtuple('Episode', field_names=['reward', 'steps'])


def iterate_batches(env, agent, net, batch_size):
  '''Generate batches of knapsack solutions. Each solution consist of the 
     steps = [observation, actions] taken to solve the problem. 
     The batch collects several solutions (batch_size) '''  
  batch = []
  episode_reward = 0.0
  episode_steps = []
  sm = nn.Softmax(dim=1)
  env.reset()

  # Repeat until enough solutions are built
  while True:
    a = agent.choose_action(env, net)     
    step_valid, is_done = env.step(a)
    if step_valid:
      step = EpisodeStep(observation=env.obs, action=a)
      episode_steps.append(step) 
    # if a solutions is complete, it reset the environment to start a new solution   
    if is_done:
      e = Episode(reward=env.kp_value, steps=episode_steps)
      batch.append(e)
      env.reset()
      episode_steps = []
      # If enough solutions are generated it returns the batch
      if len(batch) == batch_size:
        yield batch
        batch = []

## Selecciona las mejores soluciones 

Escoge las mejores soluciones de un batch y extrae las observaciones y la acción que el agente tomó en ellas

In [ ]:
def filter_batch(batch, percentile):
    '''Selects the best solutions given a percentile  ''' 
    rewards = list(map(lambda s: s.reward, batch))
    reward_bound = np.percentile(rewards, percentile)
    position_best = np.argmax(rewards)
    reward_max = rewards[position_best]    

    train_obs = []
    train_act = []
    for reward, steps in batch:
        if reward < reward_bound:
            continue
        train_obs.extend(map(lambda step: step.observation, steps))
        train_act.extend(map(lambda step: step.action, steps))
    train_obs_v = torch.FloatTensor(train_obs)
    train_act_v = torch.LongTensor(train_act)
    
    return train_obs_v, train_act_v, reward_bound, reward_max

# Experimentación

## Una instancia pequeña

In [ ]:
# Parameters
HIDDEN_SIZE = 128
BATCH_SIZE = 16
PERCENTILE = 70
N_EPISODES = 100
EPSILON = 0.01

# Lists to save the observed solutions
incunbent_hist =[0]
incunbent = 0
best_list = [0]

# Create neural network 
net = Net(n_items+2, HIDDEN_SIZE, n_items)
objective = nn.CrossEntropyLoss()
optimizer = optim.Adam(params=net.parameters(), lr=0.01)
sm = nn.Softmax(dim=1)
# Create  agent and environment
env = environment(len(ITEMS), capacity, values, weights, len(ITEMS))
ag = agent(sm, EPSILON)


# Iterate over batches. Each time a batch is created the neural network is
# updated
for iter_no, batch in enumerate(iterate_batches(env, ag, net, BATCH_SIZE)):
  
  # Filters the batch
  obs_v, acts_v, reward_b, reward_m = filter_batch(batch, PERCENTILE)
  
  # Updates the incumbent (best solution)
  if reward_m > incunbent:
    incunbent = reward_m
  incunbent_hist.append(incunbent)
  best_list.append(reward_m)

  # Update network
  optimizer.zero_grad()
  action_scores_v = net(obs_v)
  loss_v = objective(action_scores_v, acts_v)
  loss_v.backward()
  optimizer.step()

  # stops if the number of episodes is reached
  if iter_no >= N_EPISODES:
    break


Print solutions

In [ ]:
import plotly.graph_objects as go


fig = go.Figure()
x = list(range(N_EPISODES))
fig.add_trace(go.Scatter(x = x, y = incunbent_hist, name= "incumbent"))
fig.add_trace(go.Scatter(x = x, y = best_list, name= "best_episode"))
 
fig.show()

## Una instancia grande

Leemos los datos

In [ ]:
import gdown
import pandas as pd

!gdown 1eiR-X0leK1qAM2EuulKW2wdYyP2owDX4 

df_instance = pd.read_excel('knapPI_2_100_1000_1.xlsx')
df_instance
ITEMS = df_instance["id"].values.tolist()
n_items = len(ITEMS)
values = df_instance["value"].values.tolist()
weights = df_instance["weight"].values.tolist()
capacity = 1000

Corremos el algoritmo

In [ ]:
# Parameters
HIDDEN_SIZE = 128
BATCH_SIZE = 16
PERCENTILE = 70
N_EPISODES = 10000
EPSILON = 0.01

# Lists to save the observed solutions
incunbent_hist =[0]
incunbent = 0
best_list = [0]

# Create neural network 
net = Net(n_items+2, HIDDEN_SIZE, n_items)
objective = nn.CrossEntropyLoss()
optimizer = optim.Adam(params=net.parameters(), lr=0.01)
sm = nn.Softmax(dim=1)
# Create  agent and environment
env = environment(len(ITEMS), capacity, values, weights, len(ITEMS))
ag = agent(sm, EPSILON)


# Iterate over batches. Each time a batch is created the neural network is
# updated
for iter_no, batch in enumerate(iterate_batches(env, ag, net, BATCH_SIZE)):
  
  # Filters the batch
  obs_v, acts_v, reward_b, reward_m = filter_batch(batch, PERCENTILE)
  
  # Updates the incumbent (best solution)
  if reward_m > incunbent:
    incunbent = reward_m
  incunbent_hist.append(incunbent)
  best_list.append(reward_m)

  # Update network
  optimizer.zero_grad()
  action_scores_v = net(obs_v)
  loss_v = objective(action_scores_v, acts_v)
  loss_v.backward()
  optimizer.step()

  # stops if the number of episodes is reached
  if iter_no >= N_EPISODES:
    break

Gráficamos la solución

In [ ]:
import plotly.graph_objects as go


fig = go.Figure()
x = list(range(N_EPISODES))
fig.add_trace(go.Scatter(x = x, y = incunbent_hist, name= "incumbent"))
fig.add_trace(go.Scatter(x = x, y = best_list, name= "best_episode"))
 
fig.show()